### 1. 구글 드라이브와 구글 코랩 연결하기
##### (1) 연결을 위한 패키지 불러오고 경로 지정하기
- **연결하기 위해서는 데이터셋을 본인의 드라이브로 가져가는 과정이 필요**  
  공유폴더 > 데이터 공모전 폴더 > 마우스 오른쪽 버튼 > 내 드라이브에 바로가기 추가 후 사용
- 아래 코드 실행 후 url 뜨면 클릭하고 로그인하기
- 나온 KEY 값 복사해서 코드 아래 직사각형 네모창에 붙여넣고 엔터치면 완료

In [5]:
from google.colab import drive # 패키지 불러오기 
from os.path import join  

ROOT = "/content/drive"     # 드라이브 기본 경로
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # 드라이브 기본 경로 Mount

# 또는 그냥 아래와 같이 한 줄로만 해도 가능
# drive.mount('/content/drive')

# 구글 드라이브 내의 데이터 공모전 파일까지 접근하기
MY_GOOGLE_DRIVE_PATH = 'My Drive/데이터공모전/code' # 프로젝트 경로
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH) # 프로젝트 경로
print(PROJECT_PATH)

/content/drive
Mounted at /content/drive
/content/drive/My Drive/데이터공모전/code


- 경로 제대로 설정되었는지 확인하기



In [6]:
%cd "{PROJECT_PATH}"

/content/drive/My Drive/데이터공모전/code


### 2. 데이터 불러오기
- 관련 패키지 설치

In [7]:
# 데이터프레임 처리를 위한 pandas, 수학 처리를 위한 numpy
import pandas as pd
import numpy as np

# 시각화에 사용할 seaboran과 matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl  # 기본 설정
import matplotlib.font_manager as fm  # 폰트

from pylab import rcParams
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

##### (1) 데이터 변수에 넣기
- 위에서 경로를 '데이터공모전/code'까지 설정해줬기 때문에 code 이후 달라지는 경로부터 지정하면 됨
- code/data 내에 데이터셋이 들어있으니 아래처럼 data/파일명.xlsx 하면 됨
- 아래 원본데이터 변수명을 길게 쓴 이유는 서울 지역 데이터 셋만 뽑은 것을 기본값 변수로 사용하기 위함  
  ex) firm_raw: 원본 / firm: 서울 지역 데이터

In [9]:
# 데이터 불러오기: 데이터 비공개로 on하지 않음
# firm_raw = pd.DataFrame(pd.read_excel('data/기업통계_금융보안원2.xlsx'))
# new_firm_raw = pd.DataFrame(pd.read_excel('data/신설법인_금융보안원.xlsx'))
# food_raw = pd.DataFrame(pd.read_csv('data/KB국민카드_금융빅데이터경진대회_데이터_raw2.csv'))

# 데이터가 다 불러와졌으면 아래 문구 출력하기
print("Data import is done!")

Data import is done!


### 3. 데이터 탐색 및 전처리
(1) 데이터 기본 정보 확인하기
- **신설법인데이터**

In [10]:
# new_firm_raw

(2) 필요한 데이터만 뽑기
- 서울특별시 내 데이터만 남기기

In [ ]:
new_firm = new_firm_raw[new_firm_raw['시도']=='서울']

In [ ]:
# new_firm['업종코드_10차'].value_counts()

- 업종코드 10차 데이터 불러와서 결합하기

In [ ]:
industry_code = pd.DataFrame(pd.read_excel('data/ect/업종코드_10차.xlsx'))
industry_code = industry_code.rename(columns = {'코드':'업종코드_10차'})

In [ ]:
new_firm = new_firm.join(industry_code.set_index('업종코드_10차')['항목명'], on = '업종코드_10차')
new_firm = new_firm.join(industry_code.set_index('업종코드_10차')['세세분류항목명'], on = '업종코드_10차')
new_firm = new_firm.rename(columns={'항목명':'업종대분류명', '세세분류항목명':'업종중분류명'})

In [11]:
# new_firm.head(2)

- 설립연도 열만들기

In [ ]:
new_firm['설립연도'] = new_firm['설립일자'].astype(str).str[:4]
new_firm['개수']=1

In [ ]:
########### 송이 데이터: 법정동별 top3 음식점 업태 ###########
# pd.set_option('display.max_rows', 7000)
# food_permission_pivot = pd.DataFrame(food_permission.groupby(['구','동가로','업태구분명'], as_index=False)['업태개수'].sum())

# total = pd.DataFrame(columns=food_permission_pivot.columns)

# dongs = food_permission_pivot['동가로'].drop_duplicates(keep='first')

# for dong in dongs:
#   tmp = food_permission_pivot[food_permission_pivot['동가로'].str.contains(dong)].sort_values(by=['업태개수'], ascending=False)
#   total = total.append(tmp.head(3), ignore_index=True)

# pd.DataFrame(total.groupby(['구','동가로', '업태구분명'])['업태개수'].sum())

- 지역명 통일

In [ ]:
new_firm['행정동'] = new_firm['행정동'].str.replace('제','')

In [ ]:
new_firm.loc[(new_firm['행정동'] == '창신1동'), '동'] = '창신동'

- 가장 많은 업종대분류인 '도매 및 소매업' 제거

In [ ]:
new_firm_cv = new_firm
condition = new_firm_cv[new_firm_cv['업종대분류명'] == '도매 및 소매업'].index
new_firm = new_firm_cv.drop(condition)

- 행정동 숫자 모두 제거하고 합치기

In [ ]:
# # new_firm_h_dong = new_firm[
# new_firm['행정동'] = new_firm['행정동'].str.replace(pat='(.*)제[0-9]', repl=r'', regex=True)

# new_firm['행정동'].value_counts()
# # '(.*)li(.*)', r'\1LI\2', regex=True)

In [ ]:
# import re
# new_firm_h_dong = re.sub(r'', '+[0-9]+', '', new_firm[new_firm['행정동']])
# new_firm_h_dong

In [12]:
pd.set_option('display.max_rows', 50000)
new_firm_group = pd.DataFrame(new_firm.groupby(['구군','행정동','업종대분류명', '설립연도'], as_index=False)['개수'].sum())
# .sort_values(by=['구군','행정동']
# new_firm_pivot = new_firm.pivot(index=['시도','구군','업종중분류명','설립연월'], columns='개수')

total = pd.DataFrame(columns = new_firm_group.columns)
dongs = new_firm_group['행정동'].drop_duplicates(keep='first')

for dong in dongs:
  tmp = new_firm_group[new_firm_group['행정동'].str.contains(dong)].sort_values(by=['개수'], ascending=False)
  total = total.append(tmp.head(1), ignore_index=True)

pd.DataFrame(total.groupby(['구군','행정동','업종대분류명'])['개수'].sum())
# append(new_firm_group.head(3))

#total.append(tmp.head(3), ignore_index=True)

In [13]:
new_firm_pt = pd.pivot_table(new_firm_group,
                             index = ['구군', '행정동', '업종대분류명'],
                             columns = '설립연도',
                             values = '개수',
                             aggfunc = ['sum'])

# new_firm_pt

In [ ]:
# new_firm_group_row = len(new_firm_group)
# new_firm_group_col = len(new_firm_group.columns)
# new_firm_group_names = new_firm_group.columns.tolist()

# for i in range( new_firm_group_col ) :
#   try :
#     sns.distplot(new_firm_group[new_firm_group_names[i]], kde=False, rug=True) 
#     plt.show()
#   except :
#     pass

- 각 구별 행정동 개수 확인하기

In [ ]:
pd.set_option('display.max_rows', 7000)
new_firm_pivot = pd.DataFrame(new_firm.groupby(['구군','행정동'], as_index=False)['개수'].sum())
new_firm_pivot['개수']=1

In [ ]:
new_firm_pivot.pivot_table(index='구군', values='개수', aggfunc=sum)

In [ ]:
plt.rc('font', family='NanumBarunGothic')
plt.figure(figsize=(20,14))
sns.countplot(x='구군', data=new_firm)
plt.xticks(rotation=45)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] =20

- 서울특별시 영등포구 여의도동 동데이터로 실습

In [14]:
new_firm_yeouido = new_firm[new_firm['행정동'].str.contains('여의도', na=False)]
new_firm_yeouido.head(2)

In [15]:
new_firm_yeouido_pivot = pd.DataFrame(new_firm_yeouido.groupby(['업종대분류명', '업종중분류명'],as_index=False)['개수'].count())
new_firm_yeouido_pivot

In [16]:
new_firm_yeouido.sort_values(by="업종중분류명", ascending=False)

### 4. 지도로 시각화하기
1) 첫 번째 시도: plotly & mapbox
   - 그림이 그려지지 않음

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(new_firm, lat='위도', lon='경도', hover_name='기업명', hover_data=['시도', '구군'],
                        color_discrete_sequence=['fuchsia'], zoom=5, height=400)
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.show()

2) 두 번째 시도: json, folium
   - error 발생

In [ ]:
geo_data = 'data/ect/seoul'

In [ ]:
import json

class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        
        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}
        
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
    
        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)


if __name__ == '__main__':
    numpy_encoder = NumpyEncoder()

In [ ]:
import folium
map_seoul = folium.Map(location=[37.541, 126.986], zoom_start=11)   
map_seoul            

In [ ]:
import folium
map_seoul = folium.Map(location=[37.541, 126.986])

for i in new_firm.index:
  lat = new_firm.loc[i,'위도']
  lon = new_firm.loc[i,'경도']
  folium.CircleMarker([lat,lon],
                      radius=new_firm.loc[i,'개수'],
                      popup=new_firm.loc[i,'행정동'],
                      color = 'blue',
                      fill = True).add_to(map_seoul)
map_seoul                  

In [ ]:
# map = folium.Map(location=map_seoul, zoom_start=10)

# folium.Choropleth(
#     geo_data = geo_data,
#     data = new_firm,
#     columns('동','개수'),
#     key_on = 'feature.properties.동',
#     fill_color = 'BuPu',
#     #legend_name=''
# ).add_to(map)

3) 세 번째 시도: scatterplot

In [ ]:
sns.scatterplot(x='경도', y='위도', data=new_firm)

4) 네 번째 시도: gpd  
- 참고 링크: [GIS 공간정보 데이터의 구성 및 기본 이해하기](https://yganalyst.github.io/spatial_analysis/spatial_analysis_1/)

In [ ]:
! pip install geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point

In [ ]:
seoul_area = gpd.GeoDataFrame.from_file('data/ect/LARD_ADM_SECT_SGG_11.shp', encoding='cp949')
seoul_area
# pt_119 = pd.read_csv('data/서울시 안전센터관할 위치정보 (좌표계_ WGS1984).csv', encoding='cp949', dtype=str)

In [ ]:
new_firm['경도'] = new_firm['경도'].astype(float)
new_firm['위도'] = new_firm['위도'].astype(float)
new_firm['geometry'] = new_firm.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
new_firm = gpd.GeoDataFrame(new_firm, geometry='geometry')

new_firm.head()

In [ ]:
print(seoul_area.crs)
print(new_firm.crs)

In [ ]:
seoul_area.crs = {'init': 'epsg:5179'}
print(seoul_area.crs)

new_firm.crs = {'init':'epsg:4326'}
print(new_firm.crs)

In [ ]:
new_firm = new_firm.to_crs({'init':'epsg:5179'})
print(new_firm.crs)

In [ ]:
ax = seoul_area.plot(column="SGG_NM", figsize=(15,15), alpha=0.8)
new_firm.plot(ax=ax, marker='v', color='black', label='new_firm')
ax.set_title("Seoul", fontsize=20)
ax.set_axis_off()
plt.legend()
plt.show()